In [1]:
#Directory Structure:
# - data 
#   - class_1
#     - images
#   - class_2
#     - images
# - val
#   - direct_images
# - train.py -> to build the model (training + HP tuning)
# - test.py -> test/validate 
# - output
#   - model
#     - model_shape, model_weights 
#   - images_with_additional_output_tags 
# - lib -> to keep your classes, libraries 
# - bin -> executable of your program 

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import backend as k
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization


In [3]:
HP_filter1 = (3,3)
HP_filter2 = (2,2)
HP_epochs = 10

In [ ]:
# COnvolution_PATTERNS + Flatten + Dense(Relu) + Dense (SoftMax)

In [ ]:
# Convolution Patterns 
# Convolutions -> Normalize the vectors -> (Compress)MaxPool-> even more patterns
# (C-N-M)-> some amount of patterns
# (C-N-M) + (C-N-M) + (C-N-M) + (C-N-M) +.... + (C-N-M) + Flatten + Dense(Relu) + Dense (SoftMax)

In [9]:
class AnacondaVGG:
  @staticmethod
  def buildmodel(height, width, depth, classes):
    channel_dim = -1
    #block1
    inputshape = (height, width, depth)
    model = keras.Sequential()
    model.add(Conv2D(32, HP_filter1, padding='same', input_shape=inputshape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_filter1))
    model.add(Dropout(0.20))

    #block 2
    model.add(Conv2D(64, HP_filter1, padding='same'))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(32, HP_filter1, padding='same', input_shape=inputshape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_filter2))
    model.add(Dropout(0.20))

    #block_final
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    return model 
    


Conv weights calculation

assume-> filter size-> (3,3)

=> [[ w1a,w2a,w3a ],[ w1b,w2b,w3b ],[ w1c,w2c,w3c ]]


dense-> input_size * output_size + output_size

convolution-> filter_size * no.of.filter + no. of filters

(3,3) * 32 + 32
=> 9 * 32 + 32 => 288 [filter_per_channel]

# 3 layers of filters will be created for B,G,R

288 X 3 = 864 weights in total be generated 

outputs_size = 32

864 + 32 = 896 -> no of trainable parameters generate by conv2D

In [12]:
import cv2
from sklearn.preprocessing import LabelBinarizer
import imutils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import random

In [13]:
dataset = 'data'
modelpath = 'model/model.json'
labelpath = 'model/labels.bin'
init_lr = 1e-3
HP_batch_size = 1
HP_image_dim = (96,96,3)


In [20]:
from imutils import paths
data = []
labels = []
allimages = list(paths.list_images(dataset))

In [22]:
allimages = sorted(allimages)
allimages

['data/cat/007.jpeg',
 'data/cat/008.jpeg',
 'data/cat/009.jpeg',
 'data/mantisshrimp/001.jpeg',
 'data/mantisshrimp/002.jpeg',
 'data/mantisshrimp/003.jpeg',
 'data/platypus/004.jpeg',
 'data/platypus/005.jpeg',
 'data/platypus/006.jpeg']

In [24]:
random.seed(42)
random.shuffle(allimages)
allimages

['data/mantisshrimp/002.jpeg',
 'data/mantisshrimp/003.jpeg',
 'data/cat/007.jpeg',
 'data/platypus/006.jpeg',
 'data/cat/008.jpeg',
 'data/platypus/005.jpeg',
 'data/cat/009.jpeg',
 'data/mantisshrimp/001.jpeg',
 'data/platypus/004.jpeg']

In [25]:
import os
for impath in allimages:
  img = cv2.imread(impath)
  resized = cv2.resize(img,(HP_image_dim[0],HP_image_dim[1]) )
  imgdata = img_to_array(resized)
  data.append(imgdata)
  label = impath.split(os.path.sep)[-2]
  labels.append(label)
  

In [26]:
labels # [ ms c p]-> [ 1 0 0], [ 0 1 0]

['mantisshrimp',
 'mantisshrimp',
 'cat',
 'platypus',
 'cat',
 'platypus',
 'cat',
 'mantisshrimp',
 'platypus']

In [27]:
xlabels = np.array(labels)


In [28]:
lb = LabelBinarizer()
binlabels = lb.fit_transform(xlabels)

In [29]:
binlabels[0]

array([0, 1, 0])

In [30]:
normdata = np.array(data, dtype='float') / 255

In [32]:
trainx, testx, trainy, testy = train_test_split(normdata, binlabels,
                                                test_size=0.2, random_state=42)

In [34]:
testy

array([[0, 1, 0],
       [0, 1, 0]])

In [36]:
len(trainy)

7

In [37]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True )

In [38]:
from keras.optimizers import Adam
opt = Adam(lr=init_lr, decay = init_lr/HP_epochs)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [39]:
setlabels = set(labels)
classcount = len(setlabels)
classcount

3

In [40]:
model = AnacondaVGG.buildmodel(HP_image_dim[0],HP_image_dim[1],HP_image_dim[2],classcount)

In [41]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [42]:
history = model.fit_generator(aug.flow(trainx, trainy, batch_size=HP_batch_size),
                                       validation_data=(testx,testy),
                              steps_per_epoch= len(trainx)// HP_batch_size,
                              epochs= HP_epochs)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
7/7 [==============================] - 2s 76ms/step - loss: 1.0982 - accuracy: 0.2857 - val_loss: 1.2528 - val_accuracy: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 0s 35ms/step - loss: 1.0922 - accuracy: 0.2857 - val_loss: 1.4580 - val_accuracy: 0.0000e+00
Epoch 3/10
7/7 [==============================] - 0s 33ms/step - loss: 1.0840 - accuracy: 0.2857 - val_loss: 1.6540 - val_accuracy: 0.0000e+00
Epoch 4/10
7/7 [==============================] - 0s 33ms/step - loss: 1.0785 - accuracy: 0.4286 - val_loss: 1.8379 - val_accuracy: 0.0000e+00
Epoch 5/10
7/7 [==============================] - 0s 36ms/step - loss: 1.0745 - accuracy: 0.2857 - val_loss: 2.0526 - val_accuracy: 0.0000e+00
Epoch 6/10
7/7 [==============================] - 0s 34ms/step - loss: 1.0701 - accuracy: 0.4286 - val_loss: 2.1786 - val_accuracy: 0.0000e+00
Epoch 7/10
7/7 [==============================] - 0s 37ms/step - loss: 1.0677 - accuracy: 0.4286 - val_loss: 2.2569 - val_accuracy: 0.0000e+00

In [43]:
model.summary()
# bias=32
# no. of filters = 32
# input size-> 3X3 image, of 3 dimensions 
# 3X3X3 = 27
# 27X32 + 32 = 896

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_1 (ModuleWrap (None, None, None, 32)    896       
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, None, None, 32)    0         
_________________________________________________________________
module_wrapper_3 (ModuleWrap (None, None, None, 32)    128       
_________________________________________________________________
module_wrapper_4 (ModuleWrap (None, None, None, 32)    0         
_________________________________________________________________
module_wrapper_5 (ModuleWrap (None, None, None, 32)    0         
_________________________________________________________________
module_wrapper_6 (ModuleWrap (None, None, None, 64)    18496     
_________________________________________________________________
module_wrapper_7 (ModuleWrap (None, None, None, 64)   